# Timing Data

In [ ]:
import sys
import statistics
import numpy
from itertools import product
from timeit import default_timer as timer
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
print(sys.path)
from tqdm import tqdm
import gillespy2
from scipy import stats

In [ ]:
from gillespy2.basic_ssa_solver import BasicSSASolver
from gillespy2.ssa_c_solver import SSACSolver
from gillespy2.basic_root_solver import BasicRootSolver
from gillespy2.optimized_ssa_solver import OptimizedSSASolver
from gillespy2 import StochKitSolver
from gillespy2.example_models import *
import pickle
StochKitSolver.name = 'StochKitSolver'
SSASolver = OptimizedSSASolver(use_cython = False)
CythonSSASolver = OptimizedSSASolver(use_cython=True)
assert CythonSSASolver.name == 'CythonSSASolver'
modelList = [Example(), Trichloroethylene(), MichaelisMenten(), Schlogl()]
solverList = [SSASolver, CythonSSASolver, StochKitSolver, SSACSolver]

In [ ]:

timing_data = {}


#For some of the slower solvers, we are not worried about some of
#the minor fluctuations in runtime compared to the StochKit and C based solvers.
def number_trials(solver_name, trajectories):
    if solver_name in ['CythonSSASolver','StochKitSolver','SSACSolver'] or trajectories <= 256:
        return 30
    return 1


for model in modelList:
    timing_data[model.name] = {}
    
for model, solver in product(modelList, solverList):
    if solver == SSACSolver:
        solver = SSACSolver(model)
    timing_data[model.name][solver.name] = []
    for traj in range(12):
        medianList = []
        exterior_stats = []
        trials = number_trials(solver.name, 2 ** traj)
        for i in tqdm(range(trials), desc = f'Model: {model.name}, Solver: {solver.name}, Trajectories: {2**traj}'):
            start = timer()
            test_results = model.run(solver=solver, number_of_trajectories=2**traj)
            stop = timer()
            medianList.append(stop-start)
        median = statistics.median(medianList)
        mean = statistics.mean(medianList)
        timing_data[model.name][solver.name].append((median, mean, trials))
        with open('timing_data.pkl', 'wb') as f:
            pickle.dump(timing_data, f)

In [ ]:
print(timing_data)

In [ ]:
#Ideally, we can run the timed tests once and save the results for later in a .pkl file
import pickle
timing_data = None
with open('timing_data.pkl', 'rb') as f:
    timing_data = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
trajs = [2 ** i for i in range(12)]
line_style = {
    'StochKitSolver' : 'r*-',
    'SSACSolver' : 'g*-',
    'CythonSSASolver' : 'b*-',
    'OptimizedSSASolver' : 'k*-'    
}


for model in modelList:
    plt.figure()
    plt.title('{} Timing Results'.format(model.name))
    plt.xlabel('Trajectories')
    plt.ylabel('Seconds')
    for solver in solverList:
        if False and solver.name == 'OptimizedSSASolver':
            continue
        times = [timing_data[model.name][solver.name][i][0] for i in range(len(timing_data[model.name][solver.name]))]
        plt.plot(trajs, times, line_style[solver.name], label='{}'.format(solver.name))
    plt.legend(loc='best')
    plt.savefig('{}TimingResults.png'.format(model.name))

In [ ]:
import matplotlib.pyplot as plt
import math
%matplotlib inline
trajs = [2 ** i for i in range(12)]
for model in modelList:
    plt.figure()
    plt.title('{} Timing Results'.format(model.name))
    plt.xlabel('Trajectories')
    plt.ylabel('Log(Seconds)')
    for solver in solverList:
        times = [math.log(timing_data[model.name][solver.name][i][0]) for i in range(len(timing_data[model.name][solver.name]))]
        plt.plot(trajs, times, line_style[solver.name], label='{}'.format(solver.name))
    plt.legend(loc='best')
    plt.savefig('{}LogTimingResults.png'.format(model.name))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
trajs = [2 ** i for i in range(12)]
line_style = {
    'StochKitSolver' : 'r*-',
    'SSACSolver' : 'g*-',
    'CythonSSASolver' : 'b*-',
    'OptimizedSSASolver' : 'k*-'    
}


for model in modelList:
    plt.figure()
    plt.title('{} Speed Up Results'.format(model.name))
    plt.xlabel('Trajectories')
    plt.ylabel('Relative Speed Against Python Solver')
    for solver in solverList:
        if solver.name == 'OptimizedSSASolver':
            continue
        times = [timing_data[model.name]['OptimizedSSASolver'][i][0]/timing_data[model.name][solver.name][i][0] for i in range(len(timing_data[model.name][solver.name]))]
        plt.plot(trajs, times, line_style[solver.name], label='{}'.format(solver.name))
    plt.legend(loc='best')
    plt.savefig('{}SpeedupResults.png'.format(model.name))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
trajs = [2 ** i for i in range(12)]
line_style = {
    'StochKitSolver' : 'r*-',
    'SSACSolver' : 'g*-',
    'CythonSSASolver' : 'b*-',
    'OptimizedSSASolver' : 'k*-'    
}


for model in modelList:
    plt.figure()
    plt.title('{} Timing Results'.format(model.name))
    plt.xlabel('Trajectories')
    plt.ylabel('Seconds')
    for solver in solverList:
        if solver.name == 'OptimizedSSASolver':
            continue
        times = [timing_data[model.name][solver.name][i][0] for i in range(len(timing_data[model.name][solver.name]))]
        plt.plot(trajs, times, line_style[solver.name], label='{}'.format(solver.name))
    plt.legend(loc='best')
    plt.savefig('{}TimingResults.png'.format(model.name))